## Simulation using Gaussian distributed volumes

In [1]:
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt
from math import pi
from utils.sim_functions import complete_measurement



In [2]:
import importlib
import utils.sim_functions
importlib.reload(utils.sim_functions)

from utils.sim_functions import complete_measurement  # re-import updated function

# <span style="color:#EE6594"> *Complete Cropped signal equation - T1 // T 2* // PD & calculation of Flip Angle* </span> </br>
Using gaussian distribution in the spinal cords gray and white matter </br>

Completeing signal equation with T1 and TR block + the effect of the flip angle. <br>
Flip angle should be the ernst angle which by definition is the *arc cos* of exp(-TR/T1). Fixing TR leads to the FA needed to maximize signal from either GM or WM from the spinal cord. <br>

We can compute the fieldmap with all the field of view and then cropped to any size, the simulation will be realistic because the frequency offset effect is already portrayed from the whole body fieldmap calculated. </br>
For this, we are using a bounding box </br>
x: [100, 400] </br>
y: [50, 400] </br>
z: [673, 800] </br>

This is good enough coverage of the cervical spine from brain to C5

Don't forget to create your header in matlab!

### <span style="color:#BC8EC4"> *Guassian Cropped simulation* </span> 

In [5]:
path_to_cropped_segmentations = r"E:\msc_data\sc_qsm\Swiss_data\march_25_re_process\MR_simulations\sim_data/ratatouille_swisss_crop.nii.gz"
path_to_cropped_pd = r"E:\msc_data\sc_qsm\new_gauss_sims\sim_inputs\swiss_crop/gauss_pd_sc_swiss_crop.nii.gz"
path_to_cropped_t1 = r"E:\msc_data\sc_qsm\new_gauss_sims\sim_inputs\swiss_crop/gauss_t1_sc_swiss_crop.nii.gz"
path_to_cropped_t2s = r"E:\msc_data\sc_qsm\new_gauss_sims\sim_inputs\swiss_crop/gauss_t2s_sc_swiss_crop.nii.gz"
path_to_cropped_fm_ppm = r"E:\msc_data\sc_qsm\new_gauss_sims\sim_inputs/chi_to_fm_ppm/gauss_fm_ppm_sc_swiss_crop.nii.gz"
######### ####### # # ####### # # # # # # # # #### # # ### #
cropped_segmentations_img = nib.load(path_to_cropped_segmentations)
cropped_segmentations_data = cropped_segmentations_img.get_fdata()      
cropped_dimensions = np.array(cropped_segmentations_data.shape)
cropped_dimensions

array([301, 351, 128])

In [6]:
# Load nifti volumes
crop_pd_pw_img = nib.load(path_to_cropped_pd)
crop_pd_pw_data = crop_pd_pw_img.get_fdata()

crop_t2s_pw_img = nib.load(path_to_cropped_t2s)
crop_t2s_pw_data = crop_t2s_pw_img.get_fdata()

crop_t1_pw_img = nib.load(path_to_cropped_t1)
crop_t1_pw_data = crop_t1_pw_img.get_fdata()

# This is the outcome of giving the susceptibility phantom to Susceptibility to Fieldmap repo
crop_deltaB0_pw_img = nib.load(path_to_cropped_fm_ppm)
crop_dB0_pw_ppm_data = crop_deltaB0_pw_img.get_fdata()

In [21]:
# Acquisition parameters:
t1_gm = 984
t1_wm = 857
tr_custom= 45
tr_swiss_acq = 926
fieldStrength = 3 # Tesla

fa_custom_gm = np.rad2deg(np.arccos(np.exp(-tr_custom/t1_gm))) # Degrees
fa_swiss_acq = 44
print("Custom FA for gm: ", fa_custom_gm)
# Echo times must be in seconds
swiss_TE_list = [0.00686, 0.01314, 0.01942, 0.0257, 0.03198]
weird_TEs = [0.001, 0.002, 0.003, 0.004, 0.005, 0.010, 0.015, 0.020, 0.030, 0.040] # This provides better T2* fit but is unrealistic.

Custom FA for gm:  17.19615053947528


In [22]:
cropped_pw_mag, cropped_pw_phase = complete_measurement(crop_t1_pw_data, crop_pd_pw_data, crop_t2s_pw_data, cropped_dimensions, crop_dB0_pw_ppm_data , fa_swiss_acq, swiss_TE_list, tr_swiss_acq, fieldStrength, handedness= 'right')

Starting optimize_measurement
Flip angle used for:
sin($/alpha$):  0.6946583704589973
1-cos($/alpha$):  0.2806601996613488
Processing TE[0] = 0.00686  [s]
Using T1, T2* and PD for simulation
handedness=right
Coefficient of phase factor:  5.505606617515199j
Finished optimized_signal
mag shape: (301, 351, 128), phase_arr shape: (301, 351, 128)
Processing TE[1] = 0.01314  [s]
Using T1, T2* and PD for simulation
handedness=right
Coefficient of phase factor:  10.545724628884798j
Finished optimized_signal
mag shape: (301, 351, 128), phase_arr shape: (301, 351, 128)
Processing TE[2] = 0.01942  [s]
Using T1, T2* and PD for simulation
handedness=right
Coefficient of phase factor:  15.585842640254398j
Finished optimized_signal
mag shape: (301, 351, 128), phase_arr shape: (301, 351, 128)
Processing TE[3] = 0.0257  [s]
Using T1, T2* and PD for simulation
handedness=right
Coefficient of phase factor:  20.625960651624j
Finished optimized_signal
mag shape: (301, 351, 128), phase_arr shape: (301, 351,

In [23]:
temp_crop_pw_mag = nib.Nifti1Image(cropped_pw_mag, affine = cropped_segmentations_img.affine)
temp_crop_pw_ph = nib.Nifti1Image(cropped_pw_phase, affine = cropped_segmentations_img.affine)

cropped_pw_magnitude_outpath = r"E:\msc_data\sc_qsm\new_gauss_sims\mrsim_outpus\cropped_swiss/gauss2_crop_sim_mag_pro.nii.gz"
cropped_pw_phase_outpath = r"E:\msc_data\sc_qsm\new_gauss_sims\mrsim_outpus\cropped_swiss/gauss2_crop_sim_ph_pro.nii.gz"

nib.save(temp_crop_pw_mag, cropped_pw_magnitude_outpath)
nib.save(temp_crop_pw_ph, cropped_pw_phase_outpath)

In [30]:
#An example of what a values can rise a overflow in exponential
# This value was in the T2s volume because the gaussian distribution centered at 0.01
# Which was 'air' label created a few pixels with this offlayer values that caused the issue 
# returning infinity and hence ITK could not open the image
#np.exp(-0.008/-4.346339950507538e-06)

C:\Users\User\AppData\Local\Temp\ipykernel_83356\1727653921.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(-0.008/-4.346339950507538e-06)


inf